# ONEDRIVE_CSV

## Overview
This function retrieves a CSV file from the user's Microsoft OneDrive App Folder using the Microsoft Graph API and a global variable `graphToken` for authentication, which is set if the user is signed in. It returns the CSV contents as a 2D list, suitable for use in Excel.  It can be adapted to also load a CSV file into a pandas DataFrame to load very large CSV files from your OneDrive App Folder that would otherwise be too large to open directly in Excel, or if you prefer to not use Power Query or other manual steps to import the data.

## Usage
To use the `ONEDRIVE_CSV` function in Excel, enter it as a formula in a cell, specifying the path to your CSV file in the OneDrive App Folder:

```excel
=ONEDRIVE_CSV(file_path)
```
The file_path must be to a file in your OneDrive Boardflare for Python App Folder (e.g., `/Apps/Boardflare Python for Excel/your-file.csv`). The function uses app folder permissions as described in the [Microsoft Graph App Folder documentation](https://learn.microsoft.com/en-us/graph/onedrive-sharepoint-appfolder) and therefore files outside the app folder (e.g. elsewhere in your OneDrive) will not be accessible.  Note that this app folder is also used to store your functions in OneDrive, so you may see *.ipynb files in there as well.  You are free to add as many folders and files as you like to this app folder, and it will not interfere with the app's functionality.  You must also be logged into your OneDrive using the **Login** button next to the **OneDrive** header on the Functions tab before using this function in order for the app to set the `graphToken` global variable. If you are not logged in, the function will return an error.

## Arguments
| Parameter   | Type   | Required | Description                                                                                                                     | Example |
|-------------|--------|----------|---------------------------------------------------------------------------------------------------------------------------------|---------|
| file_path   | string | Yes      | The path to the CSV file in your OneDrive **App Folder** (e.g., `/Apps/Boardflare Python for Excel/your-file.csv`).             | "/Apps/Boardflare Python for Excel/data.csv" |

## Returns
| Returns | Type    | Description                                 | Example |
|---------|---------|---------------------------------------------|---------|
| table   | 2D list | The contents of the CSV file as a 2D array. | [["Name", "Value"], ["Alice", 10], ...] |

## Example

**Excel Formula:**
```excel
=ONEDRIVE_CSV("/Apps/Boardflare Python for Excel/data.csv")
```

**Expected Output:**
A 2D array in Excel with the CSV's contents, ready for further analysis.

In [ ]:
import requests
import csv
import io
import pandas as pd

def onedrive_csv(file_path):
    """
    Retrieves a CSV file from the user's Microsoft OneDrive App Folder using the Microsoft Graph API and a global variable `graphToken` for authentication. Returns the CSV contents as a 2D list, suitable for use in Excel. Can also be adapted to load the CSV into a pandas DataFrame for advanced analysis of large files.

    Args:
        file_path (str): The path to the CSV file in the user's OneDrive (e.g., '/Apps/Boardflare Python for Excel/your-file.csv').

    Returns:
        list: 2D list representing the CSV file's contents.

    Raises:
        Exception: If the file cannot be retrieved, parsed, or if arguments are invalid.
    """
    # Use global graphToken if available
    token = None
    try:
        token = globals()["graphToken"]
    except KeyError:
        token = None
    if not token:
        return "Microsoft Graph token is not set. Please click login button next to OneDrive in Functions tab."
    if not file_path or not isinstance(file_path, str):
        raise Exception("A valid file_path string must be provided.")

    url = f"https://graph.microsoft.com/v1.0/me/drive/root:{file_path}:/content"
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "text/csv"
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Failed to retrieve file: {response.status_code} {response.text}")

    csv_content = response.content.decode('utf-8')
    # You could also use pandas to read the CSV content into a DataFrame
    df = pd.read_csv(io.StringIO(csv_content))
    print(df.head())

    # For demo purposes, we will convert the CSV content to a 2D list
    reader = csv.reader(io.StringIO(csv_content))
    table = [row for row in reader]
    return table

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent / "test"))
from test_utils import get_graph_token

def test_demo_load_csv_onedrive():
    # Acquire token using shared utility
    token = get_graph_token()
    globals()["graphToken"] = token
    try:
        result = onedrive_csv("/Apps/Boardflare Python for Excel/data.csv")
        print("Loaded CSV data:", result)
        assert isinstance(result, list)
        assert all(isinstance(row, list) for row in result)
        assert len(result) > 0  # Must be at least one row
        # Require 2D list: all rows must be lists and all columns must be the same length
        col_lens = [len(row) for row in result]
        assert all(l == col_lens[0] for l in col_lens), "All rows must have the same number of columns (2D list)"
    except Exception as e:
        # Acceptable if token is not set or file is not accessible
        assert "Microsoft Graph token is not set" in str(e) or "Failed to retrieve file" in str(e)

def test_invalid_path():
    try:
        onedrive_csv(123)
    except Exception as e:
        assert "file_path string must be provided" in str(e)

ipytest.run('-s')

In [ ]:
examples = [
    ["/Apps/Boardflare Python for Excel/data.csv"],
]
